# Credit Risk Resampling Techniques

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# Read the CSV and Perform Basic Data Cleaning

In [32]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [33]:
# Load the data
file_path = Path('../credit_risk-master/LoanStats_2019Q1.csv/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [34]:
# Create our features
X = df.copy()

# Create our target
y = df["loan_status"]

In [35]:
X.drop('loan_status', axis=1, inplace=True)

In [36]:
X.describe()
# Bu komut, sayısal sütunlar için yukarıda belirtilen istatistiksel özet bilgilerini hesaplar ve kullanıcıya sunar

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [37]:
# Check the balance of our target values
status_val_counts = df['loan_status'].value_counts()

In [38]:
X = pd.get_dummies(X)

In [39]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [40]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [41]:
X_resampled

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,1600.0,0.0881,50.74,35964.0,24.12,0.0,0.0,20.0,0.0,1156.0,...,True,True,True,False,False,True,True,False,True,True
1,9000.0,0.0756,280.21,41000.0,16.89,1.0,0.0,16.0,0.0,11157.0,...,False,True,False,True,False,True,True,False,True,True
2,10000.0,0.1033,214.10,112000.0,17.75,0.0,0.0,14.0,0.0,16904.0,...,False,True,False,True,False,True,True,False,True,True
3,36000.0,0.1033,1167.21,120000.0,19.95,0.0,1.0,13.0,0.0,63248.0,...,True,True,False,True,False,True,True,False,True,True
4,18000.0,0.0881,570.81,51000.0,19.11,1.0,0.0,16.0,0.0,17586.0,...,False,True,False,True,False,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102727,20000.0,0.1102,435.05,117000.0,11.72,0.0,0.0,24.0,0.0,10273.0,...,True,True,True,False,False,True,True,False,True,True
102728,10000.0,0.1356,339.65,101000.0,19.13,1.0,1.0,10.0,0.0,8398.0,...,False,True,False,True,False,True,True,False,True,True
102729,21000.0,0.0702,648.62,70000.0,16.42,0.0,2.0,9.0,0.0,2684.0,...,False,True,False,True,True,False,True,False,True,True
102730,20000.0,0.1502,693.51,36000.0,28.57,0.0,1.0,13.0,0.0,13674.0,...,False,True,False,True,False,True,False,True,True,True


In [42]:
X.describe

<bound method NDFrame.describe of        loan_amnt  int_rate  installment  annual_inc    dti  delinq_2yrs  \
0        10500.0    0.1719       375.35     66000.0  27.24          0.0   
1        25000.0    0.2000       929.09    105000.0  20.23          0.0   
2        20000.0    0.2000       529.88     56000.0  24.26          0.0   
3        10000.0    0.1640       353.55     92000.0  31.44          0.0   
4        22000.0    0.1474       520.39     52000.0  18.76          0.0   
...          ...       ...          ...         ...    ...          ...   
68812    10000.0    0.1502       346.76     26000.0   9.60          0.0   
68813    12000.0    0.2727       368.37     63000.0  29.07          0.0   
68814     5000.0    0.1992       185.62     52000.0  14.86          0.0   
68815    40000.0    0.0646      1225.24    520000.0   9.96          0.0   
68816    16000.0    0.1131       350.36     72000.0   7.02          2.0   

       inq_last_6mths  open_acc  pub_rec  revol_bal  ...  issue_d

In [43]:
X_resampled

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,1600.0,0.0881,50.74,35964.0,24.12,0.0,0.0,20.0,0.0,1156.0,...,True,True,True,False,False,True,True,False,True,True
1,9000.0,0.0756,280.21,41000.0,16.89,1.0,0.0,16.0,0.0,11157.0,...,False,True,False,True,False,True,True,False,True,True
2,10000.0,0.1033,214.10,112000.0,17.75,0.0,0.0,14.0,0.0,16904.0,...,False,True,False,True,False,True,True,False,True,True
3,36000.0,0.1033,1167.21,120000.0,19.95,0.0,1.0,13.0,0.0,63248.0,...,True,True,False,True,False,True,True,False,True,True
4,18000.0,0.0881,570.81,51000.0,19.11,1.0,0.0,16.0,0.0,17586.0,...,False,True,False,True,False,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102727,20000.0,0.1102,435.05,117000.0,11.72,0.0,0.0,24.0,0.0,10273.0,...,True,True,True,False,False,True,True,False,True,True
102728,10000.0,0.1356,339.65,101000.0,19.13,1.0,1.0,10.0,0.0,8398.0,...,False,True,False,True,False,True,True,False,True,True
102729,21000.0,0.0702,648.62,70000.0,16.42,0.0,2.0,9.0,0.0,2684.0,...,False,True,False,True,True,False,True,False,True,True
102730,20000.0,0.1502,693.51,36000.0,28.57,0.0,1.0,13.0,0.0,13674.0,...,False,True,False,True,False,True,False,True,True,True


In [44]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [45]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

np.float64(0.6434737633024294)

In [46]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  72,   29],
       [7285, 9819]])

In [47]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.57      0.02      0.64      0.41       101
   low_risk       1.00      0.57      0.71      0.73      0.64      0.40     17104

avg / total       0.99      0.57      0.71      0.72      0.64      0.40     17205



### SMOTE Oversampling

In [48]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [49]:
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [50]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [51]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

np.float64(0.6562361071233409)

In [52]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   65,    36],
       [ 5663, 11441]])

In [53]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.67      0.02      0.66      0.43       101
   low_risk       1.00      0.67      0.64      0.80      0.66      0.43     17104

avg / total       0.99      0.67      0.64      0.80      0.66      0.43     17205



In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Verilerinizi uygun formata dönüştürme
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Veriyi float32 tipine dönüştürme
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 1. Lojistik Regresyon
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
log_pred = log_model.predict(X_test)
print("Lojistik Regresyon:")
print(classification_report(y_test, log_pred))

# 2. Karar Ağacı (Decision Tree)
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)
tree_pred = tree_model.predict(X_test)
print("\nKarar Ağacı:")
print(classification_report(y_test, tree_pred))

# 3. Rastgele Orman (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
print("\nRastgele Orman:")
print(classification_report(y_test, rf_pred))

# 4. Destek Vektör Makineleri (SVM)
svm_model = SVC(kernel='rbf')  # Kernel parametresi ekleyin
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
print("\nSVM:")
print(classification_report(y_test, svm_pred))

# 5. Yapay Sinir Ağı (ANN)
# X_train ve X_test'i NumPy dizilerine dönüştürme
X_train_ann = np.array(X_train)
X_test_ann = np.array(X_test)

# Yapay Sinir Ağı modelini oluşturma
ann_model = Sequential()
ann_model.add(Dense(units=64, activation='relu', input_dim=X_train_ann.shape[1]))  # İlk katman
ann_model.add(Dense(units=32, activation='relu'))  # Gizli katman
ann_model.add(Dense(units=1, activation='sigmoid'))  # Çıkış katmanı (binary classification için sigmoid)

# Modeli derleme
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
ann_model.fit(X_train_ann, y_train, epochs=10, batch_size=32, verbose=1)

# Modeli test etme
ann_pred = (ann_model.predict(X_test_ann) > 0.5).astype('int32')
print("\nYapay Sinir Ağı (ANN):")
print(classification_report(y_test, ann_pred))


Lojistik Regresyon:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       101
           1       0.99      1.00      1.00     17104

    accuracy                           0.99     17205
   macro avg       0.50      0.50      0.50     17205
weighted avg       0.99      0.99      0.99     17205


Karar Ağacı:
              precision    recall  f1-score   support

           0       0.51      0.51      0.51       101
           1       1.00      1.00      1.00     17104

    accuracy                           0.99     17205
   macro avg       0.76      0.76      0.76     17205
weighted avg       0.99      0.99      0.99     17205


Rastgele Orman:
              precision    recall  f1-score   support

           0       0.95      0.36      0.52       101
           1       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.97      0.68      0.76     17205
weighted avg       1.00

In [55]:
# --- Performans Karşılaştırması ---
models = ['Lojistik Regresyon', 'Karar Ağacı', 'Random Forest', 'SVM', 'Yapay Sinir Ağı']
accuracy_scores = [
    log_model.score(X_test, y_test),
    tree_model.score(X_test, y_test),
    rf_model.score(X_test, y_test),
    svm_model.score(X_test, y_test),
    ann_model.evaluate(X_test_ann, y_test)[1]  # ANN için doğruluk skoru
]

# Tablolu Performans Karşılaştırması
performance_df = pd.DataFrame({'Model': models, 'Doğruluk Skoru': accuracy_scores})
print("\nModellerin Performans Karşılaştırması:")
print(performance_df)

# En İyi Model
best_model_idx = accuracy_scores.index(max(accuracy_scores))
print(f"\nEn iyi performansı gösteren model: {models[best_model_idx]}")

538/538 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - accuracy: 0.9943 - loss: 0.0369

Modellerin Performans Karşılaştırması:
                Model  Doğruluk Skoru
0  Lojistik Regresyon        0.994130
1         Karar Ağacı        0.994304
2       Random Forest        0.996106
3                 SVM        0.994130
4     Yapay Sinir Ağı        0.994130

En iyi performansı gösteren model: Random Forest


# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [56]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({np.int64(0): 246, np.int64(1): 246})

In [57]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [58]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

np.float64(0.5450059739369634)

In [59]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  68,   33],
       [9976, 7128]])

In [60]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

np.float64(0.5450059739369634)

In [61]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.67      0.42      0.01      0.53      0.29       101
          1       1.00      0.42      0.67      0.59      0.53      0.27     17104

avg / total       0.99      0.42      0.67      0.58      0.53      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [62]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68460, 'low_risk': 62011})

In [63]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [64]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

np.float64(0.5450059739369634)

In [66]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

# y_pred = model.predict(X_test)
# confusion_matrix(y_test, y_pred)

# Eğer y_pred metin etiketleri içeriyorsa, bunları sayısal değerlere dönüştürün
y_pred_numeric = np.where(y_pred == 'high_risk', 1, 0)

# Şimdi confusion matrix'i yazdırabilirsiniz
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_numeric))


[[   31    70]
 [10163  6941]]


In [68]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
# print(classification_report_imbalanced(y_test, y_pred))
# 'high_risk' için 1 ve 'low_risk' için 0 kullanarak y_pred'i sayısal hale getirin
y_pred_numeric = np.where(y_pred == 'high_risk', 1, 0)

# Şimdi imbalanced classification report'u yazdırabilirsiniz
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_numeric))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.31      0.41      0.01      0.35      0.12       101
          1       0.99      0.41      0.31      0.58      0.35      0.13     17104

avg / total       0.98      0.41      0.31      0.57      0.35      0.13     17205

